In [1]:
import logging
import logging.config
logging.config.fileConfig('logging_config.ini')
from soso.job import Job
Job1 = Job(name="job1",start="2025-02-02T11:12:06",end="2025-02-02T12:12:06",priority=2,latitude=120,longitude=100,delivery="2025-03-02T12:12:06")
Job2 = Job(name="job2",start="2025-02-02T11:12:06",end="2025-02-02T12:12:06",priority=2,latitude=140,longitude=120,delivery="2025-03-02T12:12:06")

In [2]:
from soso.utils import parse_ground_stations,parse_satellites, parse_two_line_elements, print_api_result

In [3]:
from pathlib import Path
import os
from colorama import Fore
from skyfield.api import load
ts = load.timescale()
eph = load('de421.bsp')
# Define directories containing the satellites and orders data
project_dir = Path(os.path.dirname(''))
data_dir = project_dir / 'data'
order_data_dir = data_dir / 'orders'
satellite_data_dir = data_dir / 'satellites'
outage_request_data_dir = data_dir / 'outages'
ground_station_data_dir = data_dir / 'ground_stations'
two_line_elements = parse_two_line_elements(satellite_data_dir)

# Parse satellites and orders data
satellites = parse_satellites(satellite_data_dir, ts)
ground_stations = parse_ground_stations(ground_station_data_dir)

In [4]:
# Generate intervals for each satellite
from soso.interval_tree import generate_satellite_intervals
from soso.utils import SchedulerServer, run_server_and_send_request
from soso.interface import ScheduleOutput, ScheduleParameters
jobs = (Job1,Job2)
outage_requests= ()
satellite_passes = generate_satellite_intervals(
    satellites,
    jobs,
    outage_requests,
    ground_stations,
    (),
    ts,
    eph
)
satellite_intervals = satellite_passes.satellite_intervals
ground_station_passes = satellite_passes.ground_station_passes

2025-03-14 20:46:59,585 - soso.interval_tree.satellite_interval_trees - [INFO] - Making the interval trees took 0.28505992889404297 seconds


In [5]:

    params = ScheduleParameters(
        input_hash=None,
        two_line_elements=two_line_elements,
        jobs=jobs,
        ground_stations=ground_stations,
        outage_requests=outage_requests,
        ground_station_outage_requests=[]
    )
    
solution = run_server_and_send_request(params)

stdout from the server will be printed in cyan
stderr from the server will be printed in red
2025-03-14 20:47:01,272 - uvicorn.error - [INFO] - Started server process [21587]
2025-03-14 20:47:01,273 - uvicorn.error - [INFO] - Waiting for application startup.
2025-03-14 20:47:01,273 - uvicorn.error - [INFO] - Application startup complete.
2025-03-14 20:47:01,273 - uvicorn.error - [INFO] - Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
2025-03-14 20:47:07,626 - soso.api - [INFO] - Received scheduling request from 127.0.0.1:58037
2025-03-14 20:47:07,928 - soso.interval_tree.satellite_interval_trees - [INFO] - Making the interval trees took 0.29017090797424316 seconds
2025-03-14 20:47:07,930 - soso.genetic.genetic_scheduler - [INFO] - Starting genetic algorithm
2025-03-14 20:47:07,934 - soso.network_flow.network_flow_scheduler - [INFO] - Total network flow runtime (without plotting): 0.0029120445251464844
2025-03-14 20:47:07,969 - soso.bin_packing.ground_station_bin_packin

In [10]:
print_api_result(solution)

Satellite: SOSO-1
Satellite: SOSO-2
Satellite: SOSO-3
Satellite: SOSO-4
Satellite: SOSO-5
Impossible orders: []
Impossible orders from ground stations: []
Impossible orders from outages: []
Rejected orders: [job2 P2, job1 P2]
